# SIN2324, grupo de laboratorio 3D1
# Examen de la práctica 2


# Regresión logística aplicada a una tarea de openml

## 1 Preliminares

Las siguientes celdas de código leen una matriz de datos de openml y crean una partición train-test para hacer experimentos:

**Pon el ID del dataset que se indique**

In [1]:
pip install openml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.0 MB/s eta 0:00:00
  Created wheel for openml: filename=openml-0.14.1-py3-none-any.whl size=146923 sha256=019fda5e71dd6daaf6c399a0050c2f4e321f45b8a6b5517dae03c3c82738cd11
  Stored in directory: /root/.cache/pip/wheels/75/bc/fd/739778254a2881ef96b139d0aaf60c6d4f9130bb1459b48f10
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=e47648faa1f37a7622666a4a6fa85ca70044ae1ba870f80b0244b450f84baa07
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built openml liac-arff


In [2]:
import warnings; warnings.filterwarnings("ignore"); import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [3]:
data_id = 29    # Poner el id del dataset
X, y = fetch_openml(data_id=data_id, return_X_y=True, as_frame=False)
mask = ~np.isnan(X).any(axis=1); X = X[mask, :]; y = y[mask]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)

**Pregunta 1:** $\;$ Indica el nº de muestras, nº de características y nº de clases del dataset

In [4]:
# Escribe el código necesario:
# Número de muestras, características y clases en el conjunto de datos
num_muestras, num_caracteristicas = X.shape
num_clases = len(np.unique(y))
print("Número de muestras:", num_muestras)
print("Número de características:", num_caracteristicas)
print("Número de clases:", num_clases)


Número de muestras: 653
Número de características: 15
Número de clases: 2


**Respuesta 1:** $\;$ Escribe una respuesta breve.
Número de muestras: 653
Número de características: 15
Número de clases: 2

## 2 Experimento básico con regresión logística

Calcula el error de entrenamiento y de prueba de regresión logística con los valores por omisión que emplea la implementación de sklearn (usa siempre random_state=23) :

In [15]:
# Escribe el código: ha de escribir "Error de train: <error>% Error de test: <error>%"
# Usa en todo el examen 'random_state=23'
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
modelo_logistico = LogisticRegression(random_state=23).fit(X_train, y_train)
y_train_pred = modelo_logistico.predict(X_train)
y_test_pred = modelo_logistico.predict(X_test)
error_entrenamiento = 1 - accuracy_score(y_train, y_train_pred)
error_prueba = 1 - accuracy_score(y_test, y_test_pred)
print(f"Error de train: {error_entrenamiento * 100:.2f}% Error de test: {error_prueba * 100:.2f}%")


Error de entrenamiento: 0.17816091954022983
Error de prueba: 0.13740458015267176


**Pregunta 2:** ¿El algoritmo converje?

**Respuesta 2:** $\;$ Sí, con la configuración proporcionada en el código es probable que el algoritmo de regresión logística converja en la mayoría de los casos.

## 3 Comparación de solvers

Sklearn utiliza el solver 'solver=lbfgs' por omisión. Calcula el error en prueba de regresión logística con otros solvers y 'max_iter=10000':

In [10]:
# Escribe el código: para cada solver probado ha de escribir una línea
# "Error de test después de entrenar con el solver <solver>: <error>%"
solvers = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
for solver in solvers:
    modelo_logistico = LogisticRegression( random_state=23, solver=solver, max_iter=10000).fit(X_train, y_train)
    y_test_pred = modelo_logistico.predict(X_test)
    error_prueba = 1 - accuracy_score(y_test, y_test_pred)
    print(f"Error de test después de entrenar con el solver {solver}: {error_prueba * 100:.2f}%")

Error de test después de entrenar con el solver lbfgs: 8.40%
Error de test después de entrenar con el solver liblinear: 8.40%
Error de test después de entrenar con el solver newton-cg: 8.40%
Error de test después de entrenar con el solver newton-cholesky: 8.40%
Error de test después de entrenar con el solver sag: 34.35%
Error de test después de entrenar con el solver saga: 35.11%


**Pregunta 3:** $\;$ ¿El error obtenido con lbfgs coincide con el del experimento básico? Si no, ¿cómo lo explicas?

**Respuesta 3:** $\;$ No coincide, esto se debe a que podría atribuirse a la variabilidad inherente en los resultados de la regresión logística debido a factores como la inicialización aleatoria de los pesos del modelo, la sensibilidad a los datos y posibles diferencias en la configuración específica del solver.

## 4 Comparación de tolerancias

Scikit-learn utiliza una tolerancia 'tol=1e-4' por defecto. Utilizando 'max_iter=10000', calcula el error en la prueba de regresión logística con tolerancias inferiores y superiores al valor por defecto:

In [13]:
# Escribe el código: por cada tolerancia probada, ha de escribir una línea
# "Error de test con tolerancia <tol>: <error>%"
tols = [1e-4, 1e-2, 1, 1e2, 1e4]
for tol in tols:
    modelo_logistico = LogisticRegression(tol=tol, random_state=23, max_iter=10000).fit(X_train, y_train)
    err_test = 1 - accuracy_score(y_test, modelo_logistico.predict(X_test))
    print(f"Error de test con tolerancia {tol}: {err_test:.1%}")

Error de test con tolerancia 0.0001: 8.4%
Error de test con tolerancia 0.01: 8.4%
Error de test con tolerancia 1: 8.4%
Error de test con tolerancia 100.0: 17.6%
Error de test con tolerancia 10000.0: 34.4%


**Pregunta 4:** $\;$ ¿Que crees que pasa cuando la tolerancia es muy elevada?

**Respuesta 4:** $\;$ Cuando la tolerancia es muy elevada, el porcentaje de error se dispara. En valores hasta 1 mantiene su error, pero en cuanto llegamos a 100, se dispara (en este caso duplicandose)

## 5 Comparación de valores para el máximo número de iteraciones

Scikit-learn utiliza 'max_iter=100' por defecto. Sin embargo, como hicimos anteriormente, es conveniente utilizar un valor mayor que facilite la convergencia del algoritmo de entrenamiento y la interpretación de los resultados. Utilizando los mejores valores encontrados previamente para el solver y la tolerancia, calcula el error en la prueba de regresión logística con valores que incluyan 100 y 10000:

In [14]:
# Escribe el código: para cada valor de max_iter probado ha de escribir una línea
# "Error de test con max_iter <max_iter>: <error>%"
max_iters = [10, 20, 50, 100, 10000, 100000]
for max_iter in max_iters:
    modelo_logistico = LogisticRegression(random_state=23, max_iter=max_iter).fit(X_train, y_train)
    err_test = 1 - accuracy_score(y_test, modelo_logistico.predict(X_test))
    print(f"Error de test con max_iter {max_iter}: {err_test:.1%}")

Error de test con max_iter 10: 34.4%
Error de test con max_iter 20: 34.4%
Error de test con max_iter 50: 17.6%
Error de test con max_iter 100: 13.7%
Error de test con max_iter 10000: 8.4%
Error de test con max_iter 100000: 8.4%


**Pregunta 5:** $\;$ Con valores optimizados para el solver y la tolerancia, ¿cuántas iteraciones como mínimo se requieren para lograr un error mínimo en la prueba?

**Respuesta 5:** $\;$ Como mínimo necesitamos 10000 iteraciones.